# По мотивам семинара:

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [43]:
import pandas as pd
import numpy as np
import string
import re
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

morph = MorphAnalyzer()

In [44]:
train_data = pd.read_csv('/Users/alinashaymardanova/Desktop/3.1/Авто1/data/sentiment_twitter/train_sentiment_ttk.tsv', sep='\t')
test_data = pd.read_csv('/Users/alinashaymardanova/Desktop/3.1/Авто1/data/sentiment_twitter/test_sentiment_ttk.tsv', sep='\t')

### **CountVectorizer & LogisticRegression**

In [37]:
count_vectorizer = CountVectorizer()
count_vectorizer.fit(train_data.text.values) 

X_train = count_vectorizer.transform(train_data.text.values)
X_test = count_vectorizer.transform(test_data.text.values)

y_train = train_data.label.values
y_test = test_data.label.values


In [38]:
clf_log = LogisticRegression(penalty="l1", C=0.1)
clf_log.fit(X_train, y_train)

y_pred = clf_log.predict(X_test)

In [39]:
print(classification_report(y_test, y_pred))
print('Макросредняя F1 мера - ',f1_score(y_test, y_pred, average='macro'))
print('Микросредняя F1 мера - ',f1_score(y_test, y_pred, average='micro'))

             precision    recall  f1-score   support

         -1       0.69      0.59      0.64       902
          0       0.61      0.80      0.69       972
          1       0.30      0.03      0.06       180

avg / total       0.62      0.64      0.61      2054

Макросредняя F1 мера -  0.463064212113
Микросредняя F1 мера -  0.638753651412


### **TF-IDF & LogisticRegression**

In [40]:
tfidf = TfidfVectorizer()
tfidf.fit(train_data.text.values)

X_train = tfidf.transform(train_data.text.values)
X_test = tfidf.transform(test_data.text.values)

y_train = train_data.label.values
y_test = test_data.label.values

In [41]:
clf_log = LogisticRegression(penalty='l1')
clf_log.fit(X_train, y_train)

y_pred = clf_log.predict(X_test)

In [42]:
print(classification_report(y_test, y_pred))
print('Макросредняя F1 мера - ',f1_score(y_test, y_pred, average='macro'))
print('Микросредняя F1 мера - ',f1_score(y_test, y_pred, average='micro'))

             precision    recall  f1-score   support

         -1       0.70      0.69      0.70       902
          0       0.66      0.76      0.71       972
          1       0.37      0.09      0.15       180

avg / total       0.65      0.67      0.65      2054

Макросредняя F1 мера -  0.517616688856
Микросредняя F1 мера -  0.671372930867


# Попробуем улучшить:

In [45]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

+ Сначала, я решила попробовать с той же самой лемматизацией. Для того, чтобы улучшить результаты, решила очистить выборку от всего "мусора". 
    + От знаков преппинания, которые есть в *string.punctuation*, с добавлением кавычек-лапок и кавычек-ёлочек, которые встречаются в выборке, а так же добавила троеточие;
    + От ссылок. Постаралась рассмотреть все варианты: и http://, и https://, и просто www. ;
    + От обращений (начинаются с @);
    + От лишних пробелов/переносов/табуляций.

На самом деле, даже после такой чистки, в выборку могли просочиться какие-то символы. Не знаю почему, они шпионы какие-то.

In [6]:
stops = stopwords.words('russian')

In [7]:
def first_normalize(text):
    lemmas = []
    text = text.lower()
    text = re.sub('!|\#|\$|%|\&|\(|\)|\*|\+|,|-|\.|\/|\:|;|<|=|>|\?|\@|\[|\]|^|_|`|\{|\}|~|«|«|»|"|…', '', text)
    text = re.sub('(http://|https://|www.)[a-zA-z0-9]*? ', '', text)
    text = re.sub('[A-Za-z0-9]+', '', text)
    text = re.sub('\s+|\n|\t', ' ', text)
    tokens = wordpunct_tokenize(text)
    for el in tokens:
        lemmas.append(morph.parse(el)[0].normal_form)
    
    clean_lemmas = [lemma for lemma in lemmas 
                    if lemma not in stops]
    
    return '  '.join(clean_lemmas)

In [8]:
train_data['normalized'] = train_data['text'].apply(first_normalize)
test_data['normalized'] = test_data['text'].apply(first_normalize)

+ **TF-IDF**

In [10]:
tfidf = TfidfVectorizer()
tfidf.fit(train_data['normalized'].values)

X_train = tfidf.transform(train_data['normalized'].values)
X_test = tfidf.transform(test_data['normalized'].values)

y_train = train_data['label'].values
y_test = test_data['label'].values

+ **LogisticRegression**

In [13]:
clf_log = LogisticRegression()
clf_log.fit(X_train, y_train)

y_pred = clf_log.predict(X_test)

In [14]:
print(classification_report(y_test, y_pred))
print('Макросредняя F1 мера - ', f1_score(y_test, y_pred, average='macro'))
print('Микросредняя F1 мера - ', f1_score(y_test, y_pred, average='micro'))

             precision    recall  f1-score   support

         -1       0.75      0.55      0.63       902
          0       0.62      0.86      0.72       972
          1       0.52      0.13      0.21       180

avg / total       0.67      0.66      0.64      2054

Макросредняя F1 мера -  0.521260884222
Микросредняя F1 мера -  0.658227848101


Получается, что было:
+ Макросредняя F1 мера - 0.517616688856
+ Микросредняя F1 мера - 0.671372930867

А стало:
+ Макросредняя F1 мера -  0.521260884222
+ Микросредняя F1 мера -  0.658227848101

Не очень:)

+ Потом решила делать с помощью **стемминга**, потому что с лемметизацией вышло как-то не очень:(
Стемминг, по сравнению с лемматизацией, легче, потому что лемматизация опирается на словообразование. То есть она определает часть речи и применяет к слову различные способы нормализации. Стемминг же ищет флективную форму в своей таблице поиска, что значительно упрощает и ускоряет работу алгоритма. Кроме того, стемминг хорошо обрабатывает исключения, что в живой речи нам только на руку.

+ В предварительной "очистке" текста ничего не меняла.
+ Для твиттера есть отдельный токанайзер, который хорошо распознаёт смайлики. Используем его!

In [15]:
from nltk.tokenize import TweetTokenizer
tokenizer_words = TweetTokenizer()


In [16]:
from nltk.stem.snowball import SnowballStemmer 
stemmer = SnowballStemmer('russian')


In [17]:
def normalize(text):
    text = text.lower()
    text = re.sub('!|\#|\$|%|\&|\(|\)|\*|\+|,|-|\.|\/|\:|;|<|=|>|\?|\@|\[|\]|^|_|`|\{|\}|~|«|«|»|"|…','',text)
    text = re.sub('(http://|www.|https://)[a-zA-z0-9]*? ','', text)
    text = re.sub('[A-Za-z0-9]+','',text)
    text = re.sub('\s+|\n|\t',' ',text)
    stems = [stemmer.stem(token) for token in tokenizer_words.tokenize(text)]
    
    return '  '.join(stems)

In [18]:
train_data['normalized'] = train_data['text'].apply(normalize)
test_data['normalized'] = test_data['text'].apply(normalize)

+ TF-IDF и LogisticRegression

In [19]:
tfidf = TfidfVectorizer()
tfidf.fit(train_data['normalized'].values)

X_train = tfidf.transform(train_data['normalized'].values)
X_test = tfidf.transform(test_data['normalized'].values)

y_train = train_data['label'].values
y_test = test_data['label'].values

In [21]:
clf_log = LogisticRegression()
clf_log.fit(X_train, y_train)

y_pred = clf_log.predict(X_test)

In [22]:
print(classification_report(y_test, y_pred))
print('Макросредняя F1 мера - ', f1_score(y_test, y_pred, average='macro'))
print('Микросредняя F1 мера - ', f1_score(y_test, y_pred, average='micro'))

             precision    recall  f1-score   support

         -1       0.72      0.71      0.72       902
          0       0.68      0.78      0.73       972
          1       0.61      0.15      0.24       180

avg / total       0.69      0.70      0.68      2054

Макросредняя F1 мера -  0.561677964639
Микросредняя F1 мера -  0.696202531646


Красота:)

Было:
+ Макросредняя F1 мера - 0.517616688856
+ Микросредняя F1 мера - 0.671372930867

Стало:
+ Макросредняя F1 мера -  0.561677964639
+ Микросредняя F1 мера -  0.696202531646


Нужно попробовать улучшить результат и другими способами. Например, использовать другой алгоритм при обучении.

+ **DecisionTree**

In [23]:
from sklearn.tree import DecisionTreeClassifier

In [25]:
tree_clf = DecisionTreeClassifier()
tree_clf.fit(X_train, y_train)

y_pred = tree_clf.predict(X_test)

In [26]:
print(classification_report(y_test, y_pred))
print('Макросредняя F1 мера - ', f1_score(y_test, y_pred, average='macro'))
print('Микросредняя F1 мера - ', f1_score(y_test, y_pred, average='micro'))

             precision    recall  f1-score   support

         -1       0.62      0.61      0.61       902
          0       0.62      0.64      0.63       972
          1       0.17      0.17      0.17       180

avg / total       0.58      0.58      0.58      2054

Макросредняя F1 мера -  0.471131270592
Микросредняя F1 мера -  0.583252190847


Результат ухудшился.

+ **RandomForest**

In [27]:
from sklearn.ensemble import RandomForestClassifier

In [28]:
forest_clf = RandomForestClassifier()
forest_clf.fit(X_train, y_train)

y_pred = forest_clf.predict(X_test)

In [29]:
print(classification_report(y_test, y_pred))
print('Макросредняя F1 мера - ', f1_score(y_test, y_pred, average='macro'))
print('Микросредняя F1 мера - ', f1_score(y_test, y_pred, average='micro'))

             precision    recall  f1-score   support

         -1       0.66      0.66      0.66       902
          0       0.64      0.73      0.68       972
          1       0.41      0.09      0.15       180

avg / total       0.63      0.64      0.62      2054

Макросредняя F1 мера -  0.495225415658
Микросредняя F1 мера -  0.64264849075


Результат тоже не очень.

+ **SGD**

In [30]:
from sklearn import linear_model

In [32]:
sgd_clf = linear_model.SGDClassifier(random_state=6345)
sgd_clf.fit(X_train, y_train)

y_pred = sgd_clf.predict(X_test)

In [33]:
print(classification_report(y_test, y_pred))
print('Макросредняя F1 мера - ', f1_score(y_test, y_pred, average='macro'))
print('Микросредняя F1 мера - ', f1_score(y_test, y_pred, average='micro'))

             precision    recall  f1-score   support

         -1       0.71      0.77      0.74       902
          0       0.72      0.73      0.73       972
          1       0.54      0.26      0.35       180

avg / total       0.70      0.71      0.70      2054

Макросредняя F1 мера -  0.604001833634
Микросредняя F1 мера -  0.707400194742


Вот это уже лучше:)